In [1]:
import pandas as pd
from os import walk

In [13]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

['boston-listings.csv.gz',
 'los_angeles-listings.csv.gz',
 'new_york_city-listings.csv.gz']

In [14]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
dataFrame.head()

adding 3799 rows


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 38481 rows


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 50796 rows
total rows: 93076


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
3781,3781,https://www.airbnb.com/rooms/3781,20200316134736,2020-03-16,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,This is a totally separate apartment located o...,Fully separate apartment in a two apartment bu...,none,"Mostly quiet ( no loud music, no crowed sidewa...",...,f,f,super_strict_30,f,f,1,1,0,0,0.28
5506,5506,https://www.airbnb.com/rooms/5506,20200316134736,2020-03-16,**$79 Special ** Private! Minutes to center!,"Private guest room with private bath, You do n...",**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,"Private guest room with private bath, You do n...",none,"Peacful, Architecturally interesting, historic...",...,t,f,strict_14_with_grace_period,f,f,6,6,0,0,0.79
6695,6695,https://www.airbnb.com/rooms/6695,20200316134736,2020-03-16,$99 Special!! Home Away! Condo,"Comfortable, Fully Equipped private apartment...",** WELCOME *** FULL PRIVATE APARTMENT In a His...,"Comfortable, Fully Equipped private apartment...",none,"Peaceful, Architecturally interesting, histori...",...,t,f,strict_14_with_grace_period,f,f,6,6,0,0,0.89
8789,8789,https://www.airbnb.com/rooms/8789,20200316134736,2020-03-16,Curved Glass Studio/1bd facing Park,"Bright, 1 bed with curved glass windows facing...",Fully Furnished studio with enclosed bedroom. ...,"Bright, 1 bed with curved glass windows facing...",none,Beacon Hill is a historic neighborhood filled ...,...,f,f,strict_14_with_grace_period,f,f,7,7,0,0,0.35
10730,10730,https://www.airbnb.com/rooms/10730,20200316134736,2020-03-16,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",Bright one bed facing the golden dome of the S...,"Bright, spacious unit, new galley kitchen, new...",none,Beacon Hill is located downtown and is conveni...,...,f,f,strict_14_with_grace_period,f,f,7,7,0,0,0.24


In [15]:
#cleaning columns
for x in dataFrame.columns:
    print(x)
    values = dataFrame[x][pd.notna(dataFrame[x])].values
    if(len(values) > 0):
        print(values[0])
    else:
        print('NOT_VALUES')
    print('')

id
3781

listing_url
https://www.airbnb.com/rooms/3781

scrape_id
20200316134736

last_scraped
2020-03-16

name
HARBORSIDE-Walk to subway

summary
Fully separate apartment in a two apartment building. Space is perfect for an individual or a couple. There is a full bath, bedroom with queen bed, small sitting area with TV and a fully equipped kitchen with dining area. Walk or bus to subway.

space
This is a totally separate apartment located on the first floor of a 3 story building. Apartment has one bedroom with queen sized bed plus full eat-in kitchen,sitting room and full bath with tub/shower. Free off street parking for one car. SORRY NO SNOW REMOVAL SERVICE FOR CAR.  Free wi-fi  10 minute walk/5 minute bus (3 doors away) to subway station--next stop is Aquarium/Faneuil Hall in central Boston.  Guests who have stayed in our apartment have used it as a home base to explore Boston and surroundings. Others during extended stays at the apartment have commuted to  Massachusetts General Ho

In [16]:
# neighbourhood
# neighbourhood_cleansed
# neighbourhood_group_cleansed
# guests_included
# license
# is_business_travel_ready

cols_to_drop = [
    'market', 'street'
]
# [colName for colName in dataFrame.columns if colName not in cols_to_drop]


dataFrame.drop(cols_to_drop, axis=1, inplace=True)

In [17]:
#text pipeline steps
from sklearn.base import BaseEstimator, TransformerMixin

#Esta clase simplemente filtra las columnas que se le indica en el constructor
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ]
    
class TextTransformer(BaseEstimator, TransformerMixin):
    
    def __clean_text(self, x):
        for punct in "/-'":
            x = x.replace(punct, ' ')
        for punct in '&':
            x = x.replace(punct, f' {punct} ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~•' + '“”’':
            x = x.replace(punct, '')
        return x.lower()
    
    def __parseTextCols(self, x):
        finalTexts = []
        for i in x:
            if(pd.notna(i) and i not in finalTexts):
                finalTexts.append(i)
        text = self.__clean_text(' '.join(finalTexts))
        return text
    
    def fit(self, X, y = None):
        return self
    
    def transform (self, X, y = None):
        return X.apply(self.__parseTextCols, axis=1)
    
    
from sklearn.feature_extraction.text import TfidfVectorizer
class custom_Tfidf(TfidfVectorizer, TransformerMixin):
    options= {
        'fitSample': 1
    }
    def __init__(self, params, options = None):
        self.vectorizer = TfidfVectorizer(**params)
        if(options != None):
            for key in options.keys():
                self.options[key] = options[key]
        
    def fit(self, X, y = None):
        self.vectorizer.fit(X.sample(frac=self.options['fitSample']))
        return self
    
    def transform(self, X, y = None):
        print(len(X))
        return self.vectorizer.transform(X)

In [19]:
#pipeline
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

text_cols = ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"]

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')
textVectSettings = {
    'stop_words': stopwords_en,
    'max_df': 0.90,
    'min_df': .05,
    'ngram_range': (1,3),
    'max_features': 350
}

#Pasos para el pipeline Textos
text_pipeline = Pipeline(steps = [
    ( 'text_selector', FeatureSelector(text_cols) ),
    ( 'text_transformer', TextTransformer() ),
    ( 'text_vectorize',  custom_Tfidf(textVectSettings, {'fitSample':.01}))
] )

text_pipeline.fit(dataFrame)
# text_pipeline.transform(dataFrame)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\gusta/nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\gusta\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
dummy_cols = ["instant_bookable","is_business_travel_ready","cancellation_policy",
"require_guest_phone_verification",
"require_guest_profile_picture","host_response_time",
"host_is_superhost","host_has_profile_pic","host_identity_verified",
"city","state","property_type","room_type","bed_type"];

dummy_pipeline = Pipeline(steps = [
    ('dummy_selector', FeatureSelector(dummy_cols))
])

In [ ]:
# import nltk
# nltk.download('stopwords')

class customTransformer:
    
    textCols = []
    configuration = {
        'textTrainSample': .3
    }
    
    def __init__(self, columns, configuration = False):
        self.fitted = False
        from nltk.corpus import stopwords
        from sklearn.feature_extraction.text import TfidfTransformer
        from sklearn.feature_extraction.text import TfidfVectorizer
        stopwords_en = stopwords.words('english')
        
        textVectSettings = {
            'stop_words': stopwords_en,
            'max_df': 0.90,
            'min_df': .05,
            'ngram_range': (1,3),
            'max_features': 350
        }
        
        if(configuration != False):
            for key in configuration.keys():
                self.configuration[key] = configuration[key]
        self.Tfid = TfidfVectorizer(**textVectSettings)
        self.textCols = columns['textCols']
        
    def __clean_text(self, x):
        for punct in "/-'":
            x = x.replace(punct, ' ')
        for punct in '&':
            x = x.replace(punct, f' {punct} ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~•' + '“”’':
            x = x.replace(punct, '')
        return x.lower()
    
    def __parseTextCols(self, x):
        finalTexts = []
        for x in x[self.textCols]:
            if(pd.notna(x) and x not in finalTexts):
                finalTexts.append(x)
        text = self.__clean_text(' '.join(finalTexts))
        return text
    
    def fit_transform(self, x):
        self.fit(x)
        return self.transform(x)
    
    def fit(self, x):
        """
            Learn parameters from pandas dataframe.
        """
        sample = self.configuration['textTrainSample']
        texts = x.sample(frac=sample).apply(self.__parseTextCols, axis=1)
        self.Tfid.fit(texts)
        self.fitted= True
        
    def transform(self, x):
        """
            Transform pandas dataframe to matrix.
            ----
            Retruns matrix.
        """
        from scipy.sparse import hstack
        if(not self.fitted):
            raise NameError('customTransformer not fitted')
        texts = x.apply(self.__parseTextCols, axis=1)
        textVectors = self.Tfid.transform(texts)
        return hstack([textVectors])
    
    def save_pkl(slef, filename):
        """
            Save class with parameters to a .pkl file.
        """
        with open(filename, 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
    
        
cols = {
    'textCols': ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"]
}
configuration = {
    'textTrainSample': .01
}
transformer = customTransformer(cols, configuration)
transformer.fit(dataFrame)